# CPSC 368 Impact By State (KNM Neighbours)

## Loading Data and Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Impact by State import statements
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import TransformedTargetRegressor

### Impact by State

In [ ]:
USCDI = pd.read_csv('final_datasets_V1/cleaned/U.S._Chronic_Disease_Indicators_newcols.csv')
KFF2019_new = pd.read_csv("final_datasets_V1/cleaned/KFF2019_new.csv")

In [ ]:
condition_2_t = USCDI["Topic"] == "Cardiovascular Disease"
condition_2_q = USCDI["Question"] == "Coronary heart disease mortality among all people, underlying cause"
condition_2_dvu = USCDI["DataValueUnit"] == "cases per 100,000"
condition_2_sc1 = USCDI["StratificationCategory1"] == "Age"
condition_2_s1 = USCDI['Stratification1'].isin(['Age 0-44', 'Age 45-64'])
condition_2_dvt = USCDI["DataValueType"] == "Crude Rate"
# condition_2_syear = USCDI["YearStart"] == 2019
# condition_2_eyear = USCDI["YearEnd"] == 2019
condition_2_2019 = USCDI["Has2019"] == True

USCDI_state = USCDI[condition_2_t & condition_2_q & condition_2_dvu & condition_2_sc1 & condition_2_dvt & condition_2_2019]
USCDI_state = USCDI_state[USCDI_state['LocationDesc'] != 'United States'] # Drop Rows where LocationDesc is equal to United States

state_df = pd.merge(USCDI_state, KFF2019_new[['Location', 'All_Uninsured']], 
                     left_on='LocationDesc', right_on='Location', how='left')
state_df.drop(columns='Location', inplace=True)
state_df.rename(columns={'AvgDataValue': 'AvgDeathRate'}, inplace=True) #Rename DataValue Column to AvgDeathRate for better interpretability
state_df.rename(columns={'DataValueType': 'DeathRateType'}, inplace=True) #Rename DataValueType Column to DeathRateType for better interpretability
state_df.rename(columns={'DataValueUnit': 'DeathRateUnit'}, inplace=True) #Rename DataValueUnit Column to DeathRateUnit for better interpretability
state_df = state_df[["LocationDesc", "DeathRateUnit", "DeathRateType", "AvgDeathRate", "Stratification1", "All_Uninsured"]]

state_df.to_csv("final_datasets_V1/cleaned/state_df.csv", index=False)

In [ ]:
state_df_0_44 = state_df[state_df['Stratification1'] == 'Age 0-44']
state_df_45_64 = state_df[state_df['Stratification1'] == 'Age 45-64']
state_df_65 = state_df[state_df['Stratification1'] == 'Age >=65']

display(state_df.head())

Since the `state_df` contains overlapping rows with the same values for certain columns, I’ve decided to further split the data based on age stratification. This will ensure that each state has a unique uninsured rate and death rate for each age group, helping to eliminate redundancy and providing more precise and meaningful insights for the analysis.

In [ ]:
state_df.isnull().sum()

The target column, AvgDeathRate, is the only column with missing values. To handle this, we will impute the missing values using the median, as it is less sensitive to extreme outliers and better preserves the distribution of the data. Since we cannot guarantee that the data is normally distributed, the median is a stronger choice compared to the mean for filling in missing values. This approach ensures that the imputation is not influenced by outliers or skewed distributions, which could distort the data and lead to inaccurate results.

In [ ]:
sns.regplot(data=state_df_0_44, x='All_Uninsured', y='AvgDeathRate', scatter=True, lowess=True)
plt.title('Uninsured Rate vs Death Rate Age: 0-44')
plt.show()

In [ ]:
sns.regplot(data=state_df_45_64, x='All_Uninsured', y='AvgDeathRate', scatter=True, lowess=True)
plt.title('Uninsured Rate vs Death Rate Age: 44-64')
plt.show()

In [ ]:
sns.regplot(data=state_df_65, x='All_Uninsured', y='AvgDeathRate', scatter=True, lowess=True)
plt.title('Uninsured Rate vs Death Rate Age: 65+')
plt.show()

These regression plots show the scatter plot of individual data points along with the best-fit line from the regression. Since the scatter plot lacks a clear trend and the best-fit line has a shallow slope, it suggests that the features have limited predictive power for Death Rate. However, we'll explore how a more complex model, like SVR, performs.

#### Splitting the Data
To begin, we separate the features from the target variable. Then we'll split the dataset into training and test sets using an 80/20 split. Next, we divide the dataset into training and test sets. Additionally, we apply a log transformation to the target variable to reduce skewness and reduce the impact of outliers, with `np.log1p()` specifically to handle cases where `state_df_0_44["All_Uninsured"] == 0`. This transformation not only improves the model's fit but also enhances interpretability by stabilizing variance and making the relationships between variables more linear.

In [ ]:
State_X_0_44 = state_df_0_44[['All_Uninsured', 'LocationDesc']]
State_y_0_44 = np.log1p(state_df_0_44['AvgDeathRate'])

X_train_0_44, X_test_0_44, y_train_0_44, y_test_0_44 = train_test_split(State_X_0_44, State_y_0_44, test_size=0.2, random_state=42)

#### Preprocessing and Pipeline
To address the missing values in the AvgDeathRate column, we will impute the null entries with the median value. Following that, we'll use a column transformer to scale the numerical features and apply One-Hot Encoding to the location column, ensuring that the influence of each state is captured. We will then assemble these steps into a pipeline, making the process repeatable for other datasets. 

In [ ]:
y_train_imputed_0_44 = y_train_0_44.fillna(y_train_0_44.median())
y_test_imputed_0_44 = y_test_0_44.fillna(y_test_0_44.median())

preprocessor = ColumnTransformer(
    transformers=[
        ('num', make_pipeline(
            MinMaxScaler()
        ), ['All_Uninsured']),
        ('cat', make_pipeline(
            OneHotEncoder(handle_unknown='ignore')
        ), ['LocationDesc'])
    ]
)

pipe = make_pipeline(
    preprocessor,
    SVR()
)

ttr = TransformedTargetRegressor(
    regressor=pipe,
    transformer=StandardScaler()
)

#### Hyperparameter Optimization
We will now perform hyperparameter optimization to identify the best set of parameters for achieving optimal model performance.

In [ ]:
param_grid = {
    'regressor__svr__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'regressor__svr__epsilon': [0.01, 0.1, 0.2, 0.5, 0.08, 1.6, 2.4, 3.2],
    'regressor__svr__gamma': ['scale', 'auto', 0.01, 0.1, 1, 10, 100],
    'regressor__svr__kernel': ['linear', 'rbf', 'poly']
}

grid_search_0_44 = GridSearchCV(ttr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_0_44.fit(X_train_0_44, y_train_imputed_0_44)

print(grid_search_0_44.best_estimator_)

#### Remodel with Best Hyperparameters
Having identified the best hyperparameters, we will now retrain the SVR model using these values to achieve optimal performance. We will assess the model using RMSE to assess it's accuracy

In [ ]:
y_pred_0_44 = grid_search_0_44.best_estimator_.predict(X_test_0_44)

rmse_0_44 = np.sqrt(mean_squared_error(y_test_imputed_0_44, y_pred_0_44))

print(f"Best Parameters: {grid_search_0_44.best_params_}")
print(f"Test RMSE: {rmse_0_44:.2f}")

#### AvgDeathRate by States Age: 44-64
Now we'll repeat the steps with the other age stratifications

In [ ]:
State_X_45_64 = state_df_45_64[['All_Uninsured', 'LocationDesc']]
State_y_45_64 = np.log1p(state_df_45_64['AvgDeathRate'])

X_train_45_64, X_test_45_64, y_train_45_64, y_test_45_64 = train_test_split(State_X_45_64, State_y_45_64, test_size=0.2, random_state=42)

y_train_imputed_45_64 = y_train_45_64.fillna(y_train_45_64.median())
y_test_imputed_45_64 = y_test_45_64.fillna(y_test_45_64.median())

grid_search_45_64 = GridSearchCV(ttr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_45_64.fit(X_train_45_64, y_train_imputed_45_64)

y_pred_45_64 = grid_search_45_64.best_estimator_.predict(X_test_45_64)

rmse_45_64 = np.sqrt(mean_squared_error(y_test_imputed_45_64, y_pred_45_64))

print(f"Best Parameters: {grid_search_45_64.best_params_}")
print(f"Test RMSE: {rmse_45_64:.2f}")


#### AvgDeathRate by States Age: 65+

In [ ]:
State_X_65 = state_df_65[['All_Uninsured', 'LocationDesc']]
State_y_65 = np.log1p(state_df_65['AvgDeathRate'])

X_train_65, X_test_65, y_train_65, y_test_65 = train_test_split(State_X_65, State_y_65, test_size=0.2, random_state=42)

y_train_imputed_65 = y_train_65.fillna(y_train_65.median())
y_test_imputed_65 = y_test_65.fillna(y_test_65.median())

grid_search_65 = GridSearchCV(ttr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_65.fit(X_train_65, y_train_imputed_65)

y_pred_65 = grid_search_65.best_estimator_.predict(X_test_65)

rmse_65 = np.sqrt(mean_squared_error(y_test_imputed_65, y_pred_65))

print(f"Best Parameters: {grid_search_65.best_params_}")
print(f"Test RMSE: {rmse_65:.2f}")

#### Results

In [ ]:
results_df = pd.DataFrame({
    'Age Group': ['0-44', '45-64', '65+'],
    'Test RMSE': [rmse_0_44, rmse_45_64, rmse_65]
})
print(results_df)

Our models have a low RMSE in the log transformed space meaning that the predictions are close to the actual values in the dataset. Let's look at some of the support vectors that our model uses

#### Age 0-44 Support Vectors

In [ ]:
best_ttr_model_0_44 = grid_search_0_44.best_estimator_
best_pipeline_0_44 = best_ttr_model_0_44.regressor_
best_svr_model_0_44 = best_pipeline_0_44.named_steps['svr']

support_vectors_0_44 = best_svr_model_0_44.support_vectors_
encoder_0_44 = best_pipeline_0_44.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder']
encoded_columns_0_44 = encoder_0_44.get_feature_names_out(input_features=['LocationDesc'])
support_vector_indices_0_44 = best_svr_model_0_44.support_
original_data_support_vectors_0_44 = X_train_0_44.iloc[support_vector_indices_0_44]
encoded_location_desc_values_0_44 = encoder_0_44.transform(original_data_support_vectors_0_44[['LocationDesc']]).toarray()

for idx, encoded_vector in zip(support_vector_indices_0_44, encoded_location_desc_values_0_44):
    location_desc_value_0_44 = encoded_columns_0_44[encoded_vector.astype(bool)].tolist()
    print(f"Support vector {idx} (age 0-44): LocationDesc value(s) = {location_desc_value_0_44}")


#### Age 45-64 Support Vectors

In [ ]:
best_ttr_model_45_64 = grid_search_45_64.best_estimator_
best_pipeline_45_64 = best_ttr_model_45_64.regressor_
best_svr_model_45_64 = best_pipeline_45_64.named_steps['svr']

support_vectors_45_64 = best_svr_model_45_64.support_vectors_
encoder_45_64 = best_pipeline_45_64.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder']
encoded_columns_45_64 = encoder_45_64.get_feature_names_out(input_features=['LocationDesc'])
support_vector_indices_45_64 = best_svr_model_45_64.support_
original_data_support_vectors_45_64 = X_train_45_64.iloc[support_vector_indices_45_64]
encoded_location_desc_values_45_64 = encoder_45_64.transform(original_data_support_vectors_45_64[['LocationDesc']]).toarray()

for idx, encoded_vector in zip(support_vector_indices_45_64, encoded_location_desc_values_45_64):
    location_desc_value_45_64 = encoded_columns_45_64[encoded_vector.astype(bool)].tolist()
    print(f"Support vector {idx} (age 45-64): LocationDesc value(s) = {location_desc_value_45_64}")


#### Age 65+ Support Vectors

In [ ]:
best_ttr_model_65 = grid_search_65.best_estimator_
best_pipeline_65 = best_ttr_model_65.regressor_
best_svr_model_65 = best_pipeline_65.named_steps['svr']
support_vectors_65 = best_svr_model_65.support_vectors_
encoder_65 = best_pipeline_65.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder']
encoded_columns_65 = encoder_65.get_feature_names_out(input_features=['LocationDesc'])

support_vector_indices_65 = best_svr_model_65.support_
original_data_support_vectors_65 = X_train_65.iloc[support_vector_indices_65]
encoded_location_desc_values_65 = encoder_65.transform(original_data_support_vectors_65[['LocationDesc']]).toarray()

for idx, encoded_vector in zip(support_vector_indices_65, encoded_location_desc_values_65):
    location_desc_value_65 = encoded_columns_65[encoded_vector.astype(bool)].tolist()
    print(f"Support vector {idx} (age 65+): LocationDesc value(s) = {location_desc_value_65}")


Since there are many support vectors for LocationDesc in the age groups 0-44 and 45-64, this suggests that LocationDesc does not provide strong or consistent differentiation for these age groups. In other words, LocationDesc might not be a strong predictor for these age groups in the context of predicting coronary heart disease (CHD) mortality.

However, the fact that there are fewer support vectors for the 65+ age group indicates that LocationDesc might have some degree of predictive power for this group. This could suggest that geographic location plays a larger role in predicting CHD mortality for older adults.

Additionally, it is notable that Rhode Island appears as a support vector for all three age groups, which could indicate that this state has unique characteristics or patterns influencing CHD mortality across different age groups. This may suggest that certain regional factors in Rhode Island are particularly relevant to the model, regardless of the age group being analyzed.

Overall, the lack of a clear and consistent pattern across all age groups implies that there is no definitive, universal relationship between uninsured rates and CHD mortality across all U.S. states. The influence of LocationDesc on CHD mortality appears to vary by age group, with certain states, like Rhode Island, potentially playing a more significant role.